### Setting up the environment and importing data

In [2]:
import sqlite3

conn = sqlite3.connect("nominations.db")
schema = conn.execute("pragma table_info(nominations);").fetchall()
first_ten = conn.execute("select * from nominations limit 10;").fetchall()

In [3]:
## Checking the values
for row in first_ten:
    print(row)
    
for row in schema:
    print(row)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')
(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'M

### Creating the ceremonies table using executemany method

In [4]:
## First creating a list of values that have to be inserted in the table.
## Then inserting in the new table with list of data
## First deleting previous version of 'ceremonies' table.
conn.execute("drop table ceremonies;")
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

query_ceremonies = "create table ceremonies (id integer primary key, year integer, host text);"
conn.execute(query_ceremonies)
insert_query = "insert into ceremonies (Year, Host) values (?,?);"
conn.executemany(insert_query, years_hosts)

In [5]:
## Checking the ceremonies table values
schema = conn.execute("pragma table_info(ceremonies);").fetchall()
first_ten = conn.execute("select * from ceremonies limit 10;").fetchall()

print(schema)
print(first_ten)

[(0, 'id', 'integer', 0, None, 1), (1, 'year', 'integer', 0, None, 0), (2, 'host', 'text', 0, None, 0)]
[(1, 2010, 'Steve Martin'), (2, 2009, 'Hugh Jackman'), (3, 2008, 'Jon Stewart'), (4, 2007, 'Ellen DeGeneres'), (5, 2006, 'Jon Stewart'), (6, 2005, 'Chris Rock'), (7, 2004, 'Billy Crystal'), (8, 2003, 'Steve Martin'), (9, 2002, 'Whoopi Goldberg'), (10, 2001, 'Steve Martin')]


### Inserting foreing key relations in the table

In [6]:
## This has to be done each time you connect to the database
conn.execute("PRAGMA foreign_keys = ON;")

### Making the one-to-many relationship between ceremonies and nominations table.
### And remove the Year column from nominations and adding the ceremony_id instead

### But then its not possible to do major changes in SQLite like table changes.
### So only way out is to create a new table with same structure as 'nominations'.
### And then rename the new table as 'nominations' and delete old table.

In [7]:

conn.execute("drop table nominations_two;")
nominations_two = '''create table nominations_two (
        id integer primary key, 
        category text, 
        nominee text, 
        movie text, 
        character text, 
        won text, 
        ceremony_id integer,
        foreign key(ceremony_id) references ceremonies(id));'''

## Now selecting all the rows of the old table
nom_query = '''select ceremonies.id as ceremony_id, 
    nominations.category as category, nominations.nominee as nominee, 
    nominations.movie as movie, nominations.character as character, 
    nominations.won as won
    from nominations
    inner join ceremonies 
    on nominations.year == ceremonies.year;'''

conn.execute(nominations_two)

nominations_data = conn.execute(nom_query).fetchall()

### Creating the placeholder insert function
insert_nominations_two = '''insert into nominations_two (ceremony_id,
        category, nominee, movie, character, won) values (?,?,?,?,?,?);'''

conn.executemany(insert_nominations_two, nominations_data)

In [8]:
### Checking a few rows of the new table
print(conn.execute("select * from nominations_two limit 5;").fetchall())

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', '0', 1), (2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', '0', 1), (3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', '0', 1), (4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', '1', 1), (5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', '0', 1)]


In [9]:
## Deleting old table 'nominations'.
conn.execute("drop table nominations;")

## Renaming 'nominations_two' as 'nominations'.
conn.execute("alter table nominations_two rename to nominations;")

### Creating a Join Table movies_actors acting as an associative entity
### between 'movies' and 'actors'.

In [12]:
### First creating actors and movies tables.
## Delete previous versions of 'actors' and 'movies', if present.

#conn.execute("drop table actors;")
#conn.execute("drop table movies;")

actors = '''create table actors
    ( id integer primary key,
    actor text );'''

movies = '''create table movies
    ( id integer primary key,
    movie text);'''

conn.execute(actors)
conn.execute(movies)

In [ ]:
### Now creating the join table movies_actors.

movies_actors = '''create table movies_actors
    ( id integer primary key,
    movie_id integer,
    actor_id integer
    foreign key(movie_id) references movies(id),
    foreign key(actor_id) references actors(id));'''

conn.execute(movies_actors)